<a href="https://colab.research.google.com/github/nikshingadiya/jupyter-Colab/blob/main/CatsVsDogs2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import shutil

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "nikhilshingadiya" # username from the json file
os.environ['KAGGLE_KEY'] = "5ab414af1f4549c200ab8a319e8eb592"
# Provide your key from the json file

!kaggle competitions download -c dogs-vs-cats # api copied from kaggle

In [ ]:
base_dir = '/content/dogs-vs-cats'
os.mkdir(base_dir)

In [ ]:
from zipfile import ZipFile

file_name = "/content/train.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall(base_dir)
  print('done')

In [ ]:
from zipfile import ZipFile

file_name = "/content/test1.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall(base_dir)
  print('done')

In [ ]:
filenames=os.listdir("./dogs-vs-cats/train")

categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
  
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [ ]:
df.head()

In [ ]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [ ]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df,test_df =  np.split(df, 
                       [int(.6*len(df)), int(.8*len(df))])

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)


total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [ ]:
test_df.head()

In [ ]:
os.mkdir("./dogs-vs-cats/test")

In [ ]:

for i in test_df["filename"]:
    shutil.move("/content/dogs-vs-cats/train/"+i,"/content/dogs-vs-cats/test/"+i)
    
    

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./dogs-vs-cats/train/",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./dogs-vs-cats/train/", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                 "./dogs-vs-cats/test/",
                                                  x_col='filename',
                                                  y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size,
                                                  )

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [ ]:
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

In [ ]:
from datetime import *

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/CNN2/cats_and_dog_p1_1_{}.h5".
           format((datetime.now().strftime("%Y-%m-%d:%H-%M-%S"))))

In [ ]:
test_df

In [ ]:
test_generator = train_datagen.flow_from_dataframe(test_df,
                                                 "./dogs-vs-cats/test/",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)


In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

In [ ]:
predict

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [ ]:
test_df

In [ ]:
test_df.shape

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
test_labels=np.array(test_generator.classes)

In [ ]:
y_pred=np.array(test_df["category"])

In [ ]:
print(classification_report(test_labels, y_pred))